In [1]:
partition = 478

In [2]:
import sys
from train import main
from itertools import product  
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

n_tree_values = [5, 10, 15, 20]
tree_depth_values = [3, 5, 10]
batch_size_values = [256, 512, 1000]

best_score = 0
best_config = {}

for n_tree, tree_depth, batch_size in product(n_tree_values, tree_depth_values, batch_size_values):
    print(f"\nRunning: n_tree={n_tree}, tree_depth={tree_depth}, batch_size={batch_size}")
    sys.argv = [
        'train.py',
        '-dataset', f'gtd{partition}',
        '-n_class', '30',
        '-gpuid', '0',
        '-n_tree', str(n_tree),
        '-tree_depth', str(tree_depth),
        '-batch_size', str(batch_size),
        '-epochs', '200',
        '-jointly_training'
    ]
    
    _, _, _ = main()

    # Read best score from file (assumes one run per file)
    result_file = f"results/result_gtd{partition}"
    with open(result_file, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "Best Accuracy" in line:
                acc = float(line.split()[2])
                if acc > best_score:
                    best_score = acc
                    best_config = {
                        'n_tree': n_tree,
                        'tree_depth': tree_depth,
                        'batch_size': batch_size
                    }
print("\nBest hyperparameter configuration:")
print(best_config)
print(f"Best accuracy: {best_score}")



Running: n_tree=5, tree_depth=3, batch_size=256
Use gtd478 dataset
Train Epoch: 1 [0/10020 (0%)]	Loss: 3.402465
Train Epoch: 1 [2560/10020 (25%)]	Loss: 3.364299
Train Epoch: 1 [5120/10020 (50%)]	Loss: 3.325993
Train Epoch: 1 [7680/10020 (75%)]	Loss: 3.300448

Test set: Average loss: 3.2818, Accuracy: 1009/4320 (0.2336)

Train Epoch: 2 [0/10020 (0%)]	Loss: 3.280362
Train Epoch: 2 [2560/10020 (25%)]	Loss: 3.263813
Train Epoch: 2 [5120/10020 (50%)]	Loss: 3.242862
Train Epoch: 2 [7680/10020 (75%)]	Loss: 3.209268

Test set: Average loss: 3.2047, Accuracy: 1225/4320 (0.2836)

Train Epoch: 3 [0/10020 (0%)]	Loss: 3.204175
Train Epoch: 3 [2560/10020 (25%)]	Loss: 3.184187
Train Epoch: 3 [5120/10020 (50%)]	Loss: 3.163348
Train Epoch: 3 [7680/10020 (75%)]	Loss: 3.152549

Test set: Average loss: 3.1387, Accuracy: 1302/4320 (0.3014)

Train Epoch: 4 [0/10020 (0%)]	Loss: 3.140985
Train Epoch: 4 [2560/10020 (25%)]	Loss: 3.122943
Train Epoch: 4 [5120/10020 (50%)]	Loss: 3.100272
Train Epoch: 4 [7680/100

In [3]:
sys.argv = [
    'train.py',
    '-dataset', f'gtd{partition}',
    '-n_class', '30',
    '-gpuid', '0',
    '-n_tree', str(best_config['n_tree']),
    '-tree_depth', str(best_config['tree_depth']),
    '-batch_size', str(best_config['batch_size']),
    '-epochs', '200',
    '-jointly_training'
]

preds, targets, labels = main()


Use gtd478 dataset
Train Epoch: 1 [0/10020 (0%)]	Loss: 3.401129
Train Epoch: 1 [2560/10020 (25%)]	Loss: 3.349100
Train Epoch: 1 [5120/10020 (50%)]	Loss: 3.275231
Train Epoch: 1 [7680/10020 (75%)]	Loss: 3.219949

Test set: Average loss: 3.1671, Accuracy: 1791/4320 (0.4146)

Train Epoch: 2 [0/10020 (0%)]	Loss: 3.165534
Train Epoch: 2 [2560/10020 (25%)]	Loss: 3.119112
Train Epoch: 2 [5120/10020 (50%)]	Loss: 3.074105
Train Epoch: 2 [7680/10020 (75%)]	Loss: 3.046426

Test set: Average loss: 3.0192, Accuracy: 2364/4320 (0.5472)

Train Epoch: 3 [0/10020 (0%)]	Loss: 3.023423
Train Epoch: 3 [2560/10020 (25%)]	Loss: 2.985539
Train Epoch: 3 [5120/10020 (50%)]	Loss: 2.952965
Train Epoch: 3 [7680/10020 (75%)]	Loss: 2.928343

Test set: Average loss: 2.9108, Accuracy: 2612/4320 (0.6046)

Train Epoch: 4 [0/10020 (0%)]	Loss: 2.905193
Train Epoch: 4 [2560/10020 (25%)]	Loss: 2.877665
Train Epoch: 4 [5120/10020 (50%)]	Loss: 2.860166
Train Epoch: 4 [7680/10020 (75%)]	Loss: 2.848327

Test set: Average loss:

In [4]:
def plot_confusion_matrix(y_true, y_pred, labels, partition):
    cm = confusion_matrix(y_true, y_pred, labels=range(len(labels)))
    cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

    plt.figure(figsize=(18, 16))
    sns.heatmap(cm_normalized,
                annot=True,
                fmt=".2f",
                xticklabels=labels,
                yticklabels=labels,
                cmap="viridis",
                square=True,
                linewidths=0.5,
                cbar_kws={"shrink": 0.8})

    plt.title(f"Normalized Confusion Matrix (Partition gtd{partition})", fontsize=18)
    plt.xlabel("Predicted Label", fontsize=14)
    plt.ylabel("True Label", fontsize=14)
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()

    save_path = f"results/confusion_matrix_partition_gtd{partition}.png"
    plt.savefig(save_path, dpi=300)
    plt.close()

    print(f"Saved confusion matrix for partition gtd{partition} to {save_path}")



In [5]:
plot_confusion_matrix(targets, preds, labels, partition)

Saved confusion matrix for partition gtd478 to results/confusion_matrix_partition_gtd478.png
